<a href="https://colab.research.google.com/github/CagataySencan/StressAnalysis/blob/main/Reddit_Stress_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Suppor Vector Machine


In [113]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import nltk 
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
from string import punctuation
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import  GaussianNB
from sklearn.metrics import  confusion_matrix
from sklearn.preprocessing import LabelEncoder
from gensim.parsing.preprocessing import remove_stopwords
from collections import Counter

In [2]:
! pip install kaggle

In [3]:
! mkdir ~/.kaggle

In [4]:
! cp kaggle.json ~/.kaggle/

In [6]:
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
! kaggle datasets download ruchi798/stress-analysis-in-social-media

In [ ]:
! unzip stress-analysis-in-social-media.zip

In [114]:
trainData = pd.read_csv("dreaddit-train.csv")
testData = pd.read_csv(("dreaddit-test.csv"))

In [ ]:
# Eksik veri kontrolü
print("-Eğitim verisi-\n",trainData.isnull().sum(),"\n-------------\n","-Test Verisi-\n",testData.isnull().sum())

In [158]:
# Train ve test datalarımızı birleştirdik
X_Train = trainData['text']
X_Test = testData['text']
X = pd.concat((X_Train, X_Test), sort=False).reset_index(drop=True)

Y_Train = trainData['subreddit']
Y_Test = testData['subreddit']
Y =  pd.concat((Y_Train, Y_Test), sort=False).reset_index(drop=True)

In [ ]:
# Önişleme
nltk.download('wordnet')


In [ ]:
# Küçük harfe çevirme, stopwordslerden ve noktalama işaretlerinden temizleme
for i in range(X.count()) : 
   
   text = re.sub("[^a-zA-Z]",' ',X[i])
   text = text.lower()
   text = remove_stopwords(text)
   text = text.translate(str.maketrans('','', punctuation))
   X[i] = text

# Common wordlerden temizleme
cnt = Counter()
for text in X.values : 
  for word in text.split() :
    cnt[word] += 1

cnt.most_common(10)




In [169]:
FREQWORDS = set([w for (w, wc) in cnt.most_common(10)])
def remove_freqwords(text):
    return " ".join([word for word in str(text).split() if word not in FREQWORDS])

X = X.apply(lambda text: remove_freqwords(text))
